# 10-4. C++ 문자열의 모든 것(string과 string_view)

## basic_string

`std::string`은 사실 `basic_string`이라는 클래스 템플릿의 인스턴스화 버전.

In [ ]:
template <class CharT, class Traits = std::char_traits<CharT>,
            class Allocator = std::allocator<CharT> >
class basic_string;

`basic_string`은 `CharT` 타입의 객체들을 메모리에 연속적으로 저장하고, 여러가지 문자열 연산들을 지원해주는 클래스.
`CharT` 자리에 `char`이 오게 된다면 우리가 생각하는 `std::string`이 되는 것.

5가지 종류의 인스턴스화된 문자열들이 있음.
* `std::string` : `std::basic_string<char>`
* `std::wstring` : `std::basic_string<wchar_t>` : `wchar_t`는 윈도에서는 2바이트, 유닉스에서는 4바이트
* `std::u8string` : `std::basic_string<char8_t>` : `C++20`에 새로 추가. `char8_t`는 1바이트; UTF-8 문자열을 보관할 수 있음.
* `std::u16string` : `std::basic_string<char16_t>` : `char16_t`는 2바이트; UTF-16 문자열을 보관할 수 있음.
* `std::u32string` : `std::basic_string<char32_t>` : `char32_t`는 4바이트; UTF-32 문자열을 보관할 수 있음.

wchar_t의 크기가 시스템마다 달라서 확실한 2바이트, 4바이트 타입을 만들기 위해 아래 2개가 나온 것.

`Traits`는 주어진 `CharT` 문자들에 대해 기본적인 문자열 연산들을 정의해놓은 클래스. 그래서 문자열들을 어떻게 보관하는지에 대한 로직(`basic_string`)과, 문자열들을 어떻게 연산하는지에 대한 로직(`Traits`)을 분리시킬 수 있음. `Traits`에는 `<string>`에 정의된 `std::char_traits` 클래스의 인스턴스화 버전 전달. `string`의 경우 `char_traits<char>`을 사용.

* 숫자들의 순위가 알파벳보다 낮은 문자열

In [2]:
#include <cctype>
#include <iostream>
#include <string>

// char_traits의 모든 함수들은 static 함수입니다.
struct my_char_traits : public std::char_traits<char> {  // char_traits에서 제공하는 모든 멤버 함수들이 구현된 클래스가 전달되어야 한다.
    static int get_real_rank(char c) {
        // 숫자면 순위를 엄청 떨어트린다.
        if (isdigit(c)) {
            return c + 256;
        }
        return c;
    }
    
    static bool lt(char c1, char c2) {  // 문자들 간의 크기를 비교하는 함수
        return get_real_rank(c1) < get_real_rank(c2);
    }
    
    static int compare(const char *s1, const char *s2, size_t n) {  // n의 문자열 크기를 비교하는 함수
        while (n-- != 0) {
            if (get_real_rank(*s1) < get_real_rank(*s2)) {
                return -1;
            }
            if (get_real_rank(*s1) > get_real_rank(*s2)) {
                return 1;
            }
            ++s1;
            ++s2;
        }
        return 0;
    }
};

In [4]:
int main() {
    std::basic_string<char, my_char_traits> my_s1 = "1a";
    std::basic_string<char, my_char_traits> my_s2 = "a1";
    
    std::cout << "숫자의 우선순위가 더 낮은 문자열 : " << std::boolalpha << (my_s1 < my_s2) << std::endl;
    
    std::string s1 = "1a";
    std::string s2 = "a1";
    
    std::cout << "일반 문자열 : " << std::boolalpha << (s1 < s2) << std::endl;
}
main();

숫자의 우선순위가 더 낮은 문자열 : false
일반 문자열 : true


이렇게 간단히 `Traits`만 바꿔주는 것으로 좀 더 커스터마이징된 `basic_string`을 사용할 수 있다.

## 짧은 문자열 최적화(SSO)

`basic_string`이 저장하는 문자열은 수십만 바이트의 거대한 문자열일 수도 있고, 한 두 문자 정도의 짧은 문자열일 수도 있다.
거대한 문자열은 매우 드물게 저장되는 데 반해, 길이가 짧은 문자열들은 굉장히 많이 생성되고 소멸된다.
길이가 짧은 문자열을 여러 번 할당한다면 매 번 메모리 할당이 이루어져야 하므로 굉장히 비효율적일 것이다.

따라서 `basic_string`은 짧은 길이 문자열의 경우 따로 문자 데이터를 위한 메모리를 할당하는 대신에 그냥 객체 자체에 저장해버린다. 이것을 short string optimization이라 부른다.

In [ ]:
#include <iostream>
#include <string>

// 이와 같이 new를 전역 함수로 정의하면 모든 new 연산자를 오버로딩해버린다.
// (어떤 클래스의 멤버 함수로 정의하면 해당 클래스의 new만 오버로딩됨.)
void *operator new(std::size_t count) {
    std::cout << count << " bytes 할당" << std::endl;
    return malloc(count);
}

int main() {
    std::cout << "s1 생성 ---" << std::endl;
    std::string s1 = "this is a pretty long sentence!!!";
    std::cout << "s1 크기 : " << sizeof(s1) << std::endl;
    
    std::cout << "s2 생성 ---" << std::endl;
    std::string s2 = "short sentence";
    std::cout << "s2 크기 : " << sizeof(s2) << std::endl;
}

```
s1 생성 ---
34 bytes 할당
s1 크기 : 32
s2 생성 ---
s2 크기 : 32
```

`s1`에서만 메모리 할당 발생. 그 대신 문자열 객체의 크기가 32바이트로 꽤나 큼. 메모리 사용량을 조금 희생한 대신 성능 향상을 꽤함.

대부분의 주류 C++ 라이브러리(gcc의 libstdc++과 clang의 libc++)들은 어떤 방식이든 SSO를 사용하고 있음.
여담으로, C++ 11 이전에 basic_string의 구현에서는 Copy On Write라는 기법도 사용되었으나 개정된 표준에 따라 불가능해졌다.

## 문자열 리터럴 정의하기

C에서 문자열을 정의하기 위해선 아래와 같이 했다.

In [ ]:
const char *s = "hello";

// 혹은
char s[] = "hello";

C++의 경우는,

In [3]:
auto str = "hello";

근데 `str`는 `string`이 아니라 `const char *`임. 그래서 꼭 문자열을 만들어야겠다 싶으면 `string str = "Hello";`와 같이 타입을 명시해주어야 한다.

그래서 C++ 14에서 리터럴 연산자가 나옴.

In [ ]:
auto str = "hello"s;

std::string operator"" s(const char *str, std::size_t len);

`"hello"s`는 컴파일 과정에서 `operator""s("hello", 5);`로 바뀌게 된다.

In [ ]:
#include <iostream>
#include <string>
using namespace std::literals;

int main() {
    auto s1 = "hello"s;
    std::cout << "s1 길이 : " << s1.size() << std::endl;
}

다른 문자열 리터럴도 있음.

In [ ]:
std::string str = "hello";  // char[]
std::wstring wstr = L"hello";  // wchar_t[]

Raw string literal도 있음. C++ 11에 추가.

`R"/* delimiter */( /* 문자열 */ )/* delimiter */"`

In [ ]:
std::string str = R"(asdf
foo
asdf)";

## C++에서 한글 다루기

* UTF-8 : 문자를 최소 1부터 최대 4바이트로 표현한다. (문자마다 길이가 다름.)
* UTF-16 : 문자를 2 또는 4바이트로 표현한다.
* UTF-32 : 문자를 4바이트로 표현한다. (다루기가 간단!)

In [ ]:
std::u32string u32_str = U"이건 UTF-32 문자열 입니다";  // U는 UTF-32로 인코딩하라는 의미
std::cout << u32_str.size() << std::endl;

In [ ]:
std::string str = u8"이건 UTF-8 문자열 입니다";
// 근데 대부분의 시스템의 경우 굳이 u8을 안 붙여도 파일 형식이 UTF-8일 것이므로 알아서 UTF-8 문자열이 될 것이다.

그런데, UTF-8로 하면 `str[1]`과 같이 했을 때 `건`이 나오는 게 아님. `string`은 단순히 `char`의 연속이라고만 보기 때문에.

UTF-16으로 하면 얘기가 다르다. 알파벳, 한글, 한자 전부 다 2바이트로 인코딩. 이모지나 이집트 상형문자와 같이 유니코드상 높은 번호로 매핑되어 있는 애들은 4바이트로 인코딩됨.

In [ ]:
std::u16string u16_str = u"이건 UTF-16 문자열 입니다";

그래서 일반적인 문자들만 수록되어 있는 텍스트를 다룬다면 `u16string`을 사용하는 것만큼 좋은 것이 없다.
거의 대부분의 문자들이 2바이트로 인코딩될 것이므로, 모든 문자들이 원소 1개만큼씩을 사용한다. 따라서 대체로는 `u16_str.size()`가 일치할 것.

따라서 아래와 같이 한글의 초성만 분리해내는 코드를 작성할 수도 있다. 머리를 조금만 쓴다면 초성-중성-종성 분리까지 쉽게 가능하다.

In [ ]:
#include <iostream>
#include <string>

int main() {
    std::u16string u16_str = u"안녕하세용 모드에 코드에 오신 것을 환영합니다";
    std::string jaum[] = {"ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ",
                        "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ",
                         "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ"};
    for (char16_t c : u16_str) {
        // 유니코드 상에서 한글의 범위
        if (!(0xAC00 <= c && c <= 0xD7A3)) {
            continue;
        }
        // 한글은 AC00부터 시작해서 한 초성당 총 0x24C (588)개씩 있다.
        int offset = c - 0xAC00;
        int jaum_offset = offset / 0x24C;
        std::cout << jaum[jaum_offset];
    }
    // ㅇㄴㅎㅅㅇㅁㄷㅇㅋㄷㅇㅇㅅㄱㅇㅎㅇㅎㄴㄷ
}

안타깝게도 C++에서는 요즘에 나온 `Go` 언어처럼 인코딩된 문자열을 언어 단에서 간단히 처리할 수 있는 방법은 없다. 가장 편한 방법은 어떤 문자열이든 그냥 UTF-32 인코딩으로 바꿔버리면 되겠지만, 이는 메모리 사용량을 매우 증가시킨다.
다행히, `UTF8-CPP`라는, C++에서 여러 방식으로 인코딩된 문자열을 쉽게 다룰 수 있게 도와주는 라이브러리가 있다.(표준 라이브러리는 아니다.) 매우 간단함.

## string_view

어떤 함수에 문자열을 전달할 때, 문자열 읽기만 필요로 한다면 보통 `const std::string&`으로 받던지 아니면 `const char *` 형태로 받게 된다.

하지만 각각의 방식은 문제점이 있다. 먼저 `const string&`으로 받을 경우,

In [ ]:
#include <iostream>
#include <string>

void *operator new(std::size_t count) {
    std::cout << count << " bytes 할당" << std::endl;
    return malloc(count);
}

// 문자열에 "very"라는 단어가 있으면 true를 리턴함
bool contains_very(const std::string& str) {
    return str.find("very") != std::string::npos;
}

int main() {
    // 암묵적으로 std::string 객체가 불필요하게 생성된다.
    std::cout << std::boolalpha << contains_very("c++ string is very easy to use") << std::endl;
    
    std::cout << contains_very("c++ string is not easy to use") << std::endl;
}

```
31 bytes 할당
true
30 bytes 할당
false
```

문제는 `contains_very` 함수에 문자열 리터럴(`const char *`)을 전달한다면 인자는 `string`만 받을 수 있으므로 암묵적으로 `string` 객체가 생성된다는 점.

반대로 `const char *` 형태의 인자로 받으면, `string`을 함수에 직접 전달할 수 없고, `c_str()`을 `string`에서 `const char *` 주소갑을 뽑아내야 합니다.
그리고 `const char *`로 변환하는 과정에서 문자열의 길이에 대한 정보를 잃어버리므로, 함수 내부에서 문자열 길이 정보가 필요하면 매 번 다시 계산해야 한다.

그래서 `contains_very()`를 합리적으로 만들기 위해서는 `const string&`을 인자로 받는 오버로딩 하나, `const char *`을 인자로 받는 오버로딩 하나를 각각 준비해야 한다는 문제점이 있었다.

이러한 문제는 C++ 17에서 `string_view`가 도입되면서 해결되었다.

In [ ]:
#include <iostream>
#include <string>

void *operator new(std::size_t count) {
    std::cout << count << " bytes 할당" << std::endl;
    return malloc(count);
}

// 문자열에 "very"라는 단어가 있으면 true를 리턴함
bool contains_very(std::string_view str) {
    return str.find("very") != std:string_view::npos;
}

int main() {
    // string_view 생성 시에는 메모리 할당이 필요 없다.
    std::cout << std::boolalpha << contains_very("c++ string is very easy to use") << std::endl;
    
    std::cout << contains_very("c++ string is not easy to use") << std::endl;
    
    std::string str = "some long long long long long string";
    std::cout << "----------" << std::endl;
    std::cout << contains_very(str) << std::endl;
}

```
true 
false
37 bytes 할당
---------
false
```

`string_view`는 이름 그대로 문자열을 읽기만 하는 클래스. 이 때문에 `string_view`는 문자열을 소유하고 있지 않다. 그래서, 현재 보고 있는 문자열이 소멸된다면 정의되지 않은 작업(Undefined behavior)이 발생한다.

하지만 문자열을 소유하지 않고 읽기만 한다는 특성 때문에 `string_view` 객체 생성시에 메모리 할당이 불필요하다. 그냥 읽고 있는 문자열의 시작 주소값만 복사하면 되기 때문.
그리고 `const char *`을 인자로 받았을 때에 비해 `string_view`를 생성하면서 문자열 길이를 한 번만 계산하면 되므로 효율적.

`string_view`에서 제공하는 연산들은 당연히 원본 문자열을 수정하지 않는 연산들. 대표적으로 `find`와 `substr`.
`string`의 경우 `substr`이 실제로 부분 문자열을 새로 생성해야 하므로 $O(n)$으로 수행, 그러나 `string_view`의 경우 `substr`로 또 다른 `view`를 생성하므로 $O(1)$로 매우 빠르게 수행된다.

In [ ]:
#include <iostream>
#include <string>

void *operator new(std::size_t count) {
    std::cout << count << " bytes 할당" << std::endl;
    return malloc(count);
}

int main() {
    std::cout << "string -----" << std::endl;
    std::string s = "sometimes string is very slow";
    std::cout << "--------" << std::endl;
    std::cout << s.substr(0, 20) << std::endl << std::endl;
    
    std::cout << "string_view -----" << std::endl;
    std::string_view sw = s;
    std::cout << "--------" << std::endl;
    std::cout << sv.substr(0, 20) << std::endl;
}

```
string -----
30 bytes 할당
------------------
21 bytes 할당
sometimes string is

string_view -----
------------------
sometimes string is
```

`string`의 `substr`은 문자열을 새로 생성하였기에 메모리 할당이 발생했지만, `string_view`의 경우 `substr` 시에 메모리 할당이 발생하지 않았다.

물론 위 `string_view`들은 모두 `s`에서 만들어진 것이므로 `s`가 소멸되면 사용할 수 없게 된다.

In [ ]:
#include <iostream>
#include <string>

std::string_view return_sv() {
    std::string s = "this is a string";
    std::string_view sv = s;
    
    return sv;
}

int main() {
    std::string_view sv = return_sv();  // <- sv가 가리키는 s는 이미 소멸됨!
    
    // Undefined behavior!!!!
    std::cout << sv << std::endl;
}

## 뭘 배웠지?

* `std::string`은 `basic_string`의 `char`을 인자로 갖는 템플릿 인스턴스화 버전. 그 외에도 `u8string`, `u16string`, `u32string`이 있고 각각은 UTF-8, UTF-16, UTF-32으로 인코딩된 문자열을 보관할 수 있다.
* `std::char_traits`를 사용해서 사용자가 원하는 기능을 가진 문자열을 생성할 수 있다.
* 유니코드는 전 세계의 모든 문자들을 컴퓨터에서 표현하고자 각각의 문자에 대해 고유의 코드를 부여한 것. 코드 그대로 저장하려면 4바이트가 필요한데, 이는 매우 비효율적이므로 여러가지 인코딩을 통해서 크기를 줄일 수 있다. 하지만 이 때문에 문자별로 인코딩되는 길이가 다르다는 문제점이 있다.
* `string_view`를 통해서 불필요한 복사를 막고 `const char *`과 `const string&` 사이에서 깔끔하게 처리할 수 있다.